In [16]:
import pandas as pd
import pathlib

#DS='ml1m'
DS='ft'
#DS='anime'
#DS='netflix'

EXPERIMENT='groups_2022_06_complete_varios_ds'

EXPERIMENTPATH=EXPERIMENT+'/'+DS+'/'
DATAPATH='../rs-data-python/grupos/'+DS+'/'
#groups-4-biasedMF_k8_dsml1m_seed1234_indi_expert.csv
#experiment_prefix='groups-4-'
experiment_subfix='_k8_ds'+DS+'_seed1234'

models_dl=[
    'gmf',
    'mlp',
    #'neumf',
    #'evodeep',
]

models_ml = [
    'biasedmf-avg',
    'pred-mfgu_af',
    'pred-mfgu_bf'
]

In [17]:
# DL MAEs

pd_maes = pd.DataFrame(columns=['mae'], index=models_dl)

for m in models_dl:
    with open(f'{EXPERIMENTPATH}{m}{experiment_subfix}.mae.result') as f:
        d = f.readlines()[0].split(';')[0]
    pd_maes['mae'][m]=d

pd_maes

,mae
gmf,0.6534254550933838
mlp,0.6336560845375061


In [18]:
# Reading groups

groups = {}

fromngroups=2
tongroups=10
#fromngroups=4
#tongroups=4

for g in range(fromngroups,tongroups+1):
    groups[g] = pd.read_csv(f"{DATAPATH}groups-{g}.csv")
    groups[g]['g'] = g

In [19]:
groups[4]

,item,user-1,rating-1,user-2,rating-2,user-3,rating-3,user-4,rating-4,g
0,79,162,1.0,75,3.0,21,1.0,84,1.0,4
1,57,240,2.0,349,2.5,31,2.5,396,3.5,4
2,74,71,4.0,241,4.0,103,3.0,7,4.0,4
3,158,392,4.0,341,3.5,228,2.5,103,3.0,4
4,71,3,0.5,331,3.5,341,4.0,75,4.0,4
...,...,...,...,...,...,...,...,...,...,...
49995,83,84,4.0,393,4.0,75,2.0,103,3.0,4
49996,137,330,4.0,75,3.0,405,1.5,162,3.5,4
49997,161,103,3.0,75,4.0,98,4.0,341,3.5,4
49998,161,75,4.0,103,3.0,341,3.5,98,4.0,4


In [20]:
# Reading Model results

dl_variations_list = [
                         '_indi_mean',
                         #'_indi_expert',
                         '_mo',
                         '_mo_expert',
                         #'_inverse_expert_info', 
                         #'_max_expert', '_min_expert',
                         '_softmax',
                         #'_item',
                         #'_max',
                         #'_inverse_softmax',
                         #'_raro', '_softmax_raro'
]

#ml_variations = [ m+experiment_subfix+v for m in models_ml for v in ['_indi_mean', '_indi_expert'] ]
ml_variations = models_ml
dl_variations = [ m+experiment_subfix+v for m in models_dl for v in dl_variations_list ]
#model_files = ml_variations + dl_variations
model_files = dl_variations

def shorten_name(ori, subfix):
    first_index = ori.index('_')
    second_index = ori.index('_',first_index + len(subfix))
    return ori.replace(ori[first_index:second_index],'')

models = [shorten_name(m, experiment_subfix) for m in model_files]
for g in range(fromngroups,tongroups+1):
    for mname, m in zip(models, model_files):
        # short names
        groups[g][mname] = pd.read_csv(
            f"{EXPERIMENTPATH}groups-{g}-{m}.csv",
            header=0,
            names=["data"]
        )['data']

In [21]:
list(groups[4].columns)

['item',
 'user-1',
 'rating-1',
 'user-2',
 'rating-2',
 'user-3',
 'rating-3',
 'user-4',
 'rating-4',
 'g',
 'gmf_indi_mean',
 'gmf_mo',
 'gmf_mo_expert',
 'gmf_softmax',
 'mlp_indi_mean',
 'mlp_mo',
 'mlp_mo_expert',
 'mlp_softmax']

In [22]:
# Individual diff

for g in range(fromngroups,tongroups+1):    # Each group
    for m in models:                        # Each model
        for gi in range(1,g+1):             # Each rating in group
            groups[g][m+'-error-'+str(gi)] = abs(groups[g]['rating-'+str(gi)] - groups[g][m])
            groups[g][m+'-error2-'+str(gi)] = (groups[g]['rating-'+str(gi)] - groups[g][m])**2

In [23]:
# MAE diffs
for g in range(fromngroups,tongroups+1):    # Each group
    for m in models:                        # Each model
        groups[g][m+'-MAE'] = groups[g].filter(regex='^'+m+'-error-',axis=1).mean(axis=1, skipna=True)
        #groups[g][m+'-STD'] = groups[g].filter(regex='^'+m+'-error-',axis=1).std(axis=1, skipna=True)
        groups[g][m+'-MAX'] = groups[g].filter(regex='^'+m+'-error-',axis=1).max(axis=1, skipna=True)
        groups[g][m+'-MSE'] = groups[g].filter(regex='^'+m+'-error2-',axis=1).mean(axis=1, skipna=True)


In [24]:
from lets_plot import *
LetsPlot.setup_html()

In [25]:
# Make a unique Dataframe, remove groups individual rating
all_data = pd.concat(groups, join='inner', ignore_index=True).drop(
    ['item', 'user-1', 'rating-1', 'user-2', 'rating-2'], axis=1
)
all_data

,g,gmf_indi_mean,gmf_mo,gmf_mo_expert,gmf_softmax,mlp_indi_mean,mlp_mo,mlp_mo_expert,mlp_softmax,gmf_indi_mean-error-1,...,mlp_indi_mean-MSE,mlp_mo-MAE,mlp_mo-MAX,mlp_mo-MSE,mlp_mo_expert-MAE,mlp_mo_expert-MAX,mlp_mo_expert-MSE,mlp_softmax-MAE,mlp_softmax-MAX,mlp_softmax-MSE
0,2,3.5324,3.5324,3.4441,3.4888,3.6750,3.6126,3.5704,3.5918,1.0324,...,0.7056,0.6126,1.1126,0.6253,0.5704,1.0704,0.5754,0.5918,1.0918,0.6002
1,2,3.2112,3.2112,3.1409,3.1763,3.0902,3.2483,3.0631,3.1919,1.2112,...,1.0081,1.0000,1.2483,1.0617,1.0000,1.0631,1.0040,1.0000,1.1919,1.0368
2,2,3.8029,3.8029,3.7449,3.7742,3.4937,3.4453,3.4029,3.4243,0.3029,...,1.1156,0.7500,1.4453,1.0459,0.7500,1.4029,0.9888,0.7500,1.4243,1.0172
3,2,2.2300,2.2300,2.2565,2.2425,2.7759,2.7683,2.8984,2.8280,1.2700,...,1.0114,0.9817,1.2317,1.0262,0.8516,1.1016,0.7877,0.9220,1.1720,0.9127
4,2,2.5465,2.5465,2.5616,2.5541,3.2664,3.2839,3.2940,3.2889,1.5465,...,2.8618,1.5339,2.2839,2.9154,1.5440,2.2940,2.9465,1.5389,2.2889,2.9308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,10,3.5259,3.5259,3.4136,3.5140,3.6273,3.5107,3.4357,3.5028,0.4741,...,1.4199,0.9085,2.0107,1.2172,0.8486,1.9357,1.1013,0.9023,2.0028,1.2045
449996,10,3.0792,3.0792,3.0429,3.0755,3.2180,3.1100,3.2032,3.1190,0.4208,...,1.1039,0.8000,2.6100,1.0901,0.8000,2.7032,1.1007,0.8000,2.6190,1.0904
449997,10,2.6229,2.6229,2.6126,2.6219,2.5158,2.5084,2.5043,2.5080,0.8771,...,0.9174,0.6517,2.0084,0.9209,0.6509,2.0043,0.9229,0.6516,2.0080,0.9211
449998,10,3.0184,3.0184,3.0537,3.0218,3.3199,3.0556,3.1701,3.0668,0.9816,...,0.9314,0.8389,2.0556,0.9114,0.8160,2.1701,0.9029,0.8366,2.0668,0.9094


In [26]:
def table_describe(what):
    pd.set_option("display.precision", 4)
    
    focus_on_data = [
        'g',
        'gmf_indi_mean-'+what,
        #'gmf_mo-'+what,
        #'gmf_max-'+what,
        'gmf_mo_expert-'+what,
        'gmf_softmax-'+what,
        'mlp_indi_mean-'+what,
        'mlp_mo-'+what,
        #'mlp_max-'+what,
        'mlp_mo_expert-'+what,
        'mlp_softmax-'+what,
    ]
    names = [
        'g',
        #'GMF IPA',
        #'GMF Avg',
        '\\makecell[l]{GMF IPA \\\\ GMF Avg}',
        #'GMF Most expert',
        'GMF Expertise',
        'GMF Softmax',
        'MLP IPA',
        '\\makecell[l]{MLP Avg \\\\ DeepGroup}',
        #'MLP Most expert',
        'MLP Expertise',
        'MLP Softmax',
    ]
    d = all_data.copy()[focus_on_data]
    new_names_map = {d.columns[i]:names[i] for i in range(len(names))}
    d.rename(new_names_map, axis=1, inplace=True)
    std = d.groupby("g", as_index=True).std().T
    mean = d.groupby("g", as_index=True).mean().T
    return (mean, std)


def generate_latext_table(mean, std):
    content = ""
    
    content += "\\rline{2-9}\n"
    content += "\\cline{2-9}\n"
    content += "&  \\SetCell[c=9]{c} Group Size \\\\  \\hline\n"
    content += "Model & 2      & 3      & 4      & 5      & 6      & 7      & 8      & 9      & 10     \\\\ \\hline\n"
    
    for rix in range(len(mean)):
        content += mean.index[rix]
        for cix in range(len(mean.iloc[rix])):
            cell = " & \\makecell{"
            cell += str(round(mean.iloc[rix, cix], 4))
            cell += "\\\\(" + str(round(std.iloc[rix, cix], 3)) + ")"
            cell += "} "
            content += cell
        content += "\\\\ \\hline \n"
    return content

def generate_highlight_table(mean):
    #return mean.index.get_loc(mean[3].idxmin())
    index_color = ""
    min_values = mean.idxmin()
    for i, r_label in enumerate(min_values):
        rix = mean.index.get_loc(r_label)
        index_color += "cell{"+str(rix+3)+"}{"+str(i+2)+"} = {gray9,font=\\bfseries},\n"
    return index_color



#print(generate_highlight_table(mean))
#print(generate_latext_table(mean, std))

In [27]:
file_name = "tabresults" + DS + ".txt"
content = """
\\documentclass[sn-article.tex]{subfiles}
\\usepackage{makecell}
\\begin{document}

\\begin{table}[ht]
    \\caption{"""+DS+"""}
    \\label{tab:"""+DS+"""}
"""

for w in ["MAE", "MSE", "MAX"]:
    mean, std = table_describe(w)
    
    content += """
\\begin{subtable}[h]{1\\textwidth}
\\resizebox{\\textwidth}{!}{%
    \\begin{tblr}{
        colspec = {l|c|c|c|c|c|c|c|c|c|},"""
    content += generate_highlight_table(mean)
    content +="""}
    """
    content += generate_latext_table(mean, std)
    content += """
    \\end{tblr}%
}
\\caption{"""+w+"""}
\\label{tab:"""+DS+""":"""+w+"""}
\\end{subtable}
    """
    

content += """
\\end{table}

\\end{document}
"""

print(content)
f = open(file_name, "w")
f.write(content)
f.close()


\documentclass[sn-article.tex]{subfiles}
\usepackage{makecell}
\begin{document}

\begin{table}[ht]
    \caption{ft}
    \label{tab:ft}

\begin{subtable}[h]{1\textwidth}
\resizebox{\textwidth}{!}{%
    \begin{tblr}{
        colspec = {l|c|c|c|c|c|c|c|c|c|},cell{5}{2} = {gray9,font=\bfseries},
cell{6}{3} = {gray9,font=\bfseries},
cell{6}{4} = {gray9,font=\bfseries},
cell{6}{5} = {gray9,font=\bfseries},
cell{9}{6} = {gray9,font=\bfseries},
cell{7}{7} = {gray9,font=\bfseries},
cell{6}{8} = {gray9,font=\bfseries},
cell{6}{9} = {gray9,font=\bfseries},
cell{8}{10} = {gray9,font=\bfseries},
}
    \rline{2-9}
\cline{2-9}
&  \SetCell[c=9]{c} Group Size \\  \hline
Model & 2      & 3      & 4      & 5      & 6      & 7      & 8      & 9      & 10     \\ \hline
\makecell[l]{GMF IPA \\ GMF Avg} & \makecell{0.7377\\(0.483)}  & \makecell{0.6814\\(0.405)}  & \makecell{0.6725\\(0.377)}  & \makecell{0.6639\\(0.28)}  & \makecell{0.6482\\(0.275)}  & \makecell{0.6585\\(0.272)}  & \makecell{0.6484\\(0.233)}

In [ ]:

what='MAE'
focus_on_data = [
    'g',
    'gmf_item-'+what,
    'gmf_max-'+what,
    'gmf_indi_mean-'+what,
    'gmf_mo-'+what,
    'gmf_mo_expert-'+what,
    'gmf_softmax-'+what,
    'mlp_item-'+what,
    'mlp_max-'+what,
    'mlp_indi_mean-'+what,
    'mlp_mo-'+what,
    'mlp_mo_expert-'+what,
    'mlp_softmax-'+what,
]

pd.set_option("display.precision", 4)
all_data[focus_on_data].groupby("g", as_index=False).mean().filter(regex=what+"$").T\
                .style.highlight_min(color = 'lightgreen', axis = 0)
#print(all_data[focus_on_data].groupby("g", as_index=False).mean().filter(regex=what+"$").T.to_latex())

In [ ]:
print(all_data[focus_on_data].groupby("g", as_index=False).mean().filter(regex=what+"$").T.to_latex())

In [ ]:
what='STD'
focus_on_data = [
    'g',
    'gmf_item-'+what,
    'gmf_max-'+what,
    'gmf_indi_mean-'+what,
    'gmf_mo-'+what,
    'gmf_mo_expert-'+what,
    'gmf_softmax-'+what,
    'mlp_item-'+what,
    'mlp_max-'+what,
    'mlp_indi_mean-'+what,
    'mlp_mo-'+what,
    'mlp_mo_expert-'+what,
    'mlp_softmax-'+what,
]
pd.set_option("display.precision", 4)
all_data[focus_on_data].groupby("g", as_index=False).mean().filter(regex=what+"$").T\
                .style.highlight_min(color = 'lightgreen', axis = 0)

In [ ]:
what='MAX'
focus_on_data = [
    'g',
    'gmf_item-'+what,
    'gmf_max-'+what,
    'gmf_indi_mean-'+what,
    'gmf_mo-'+what,
    'gmf_mo_expert-'+what,
    'gmf_softmax-'+what,
    'mlp_item-'+what,
    'mlp_max-'+what,
    'mlp_indi_mean-'+what,
    'mlp_mo-'+what,
    'mlp_mo_expert-'+what,
    'mlp_softmax-'+what,
]

pd.set_option("display.precision", 4)
all_data[focus_on_data].groupby("g", as_index=False).mean().filter(regex=what+"$").T\
                .style.highlight_min(color = 'lightgreen', axis = 0)

In [ ]:
what='MSE'
focus_on_data = [
    'g',
    'gmf_item-'+what,
    'gmf_max-'+what,
    'gmf_indi_mean-'+what,
    'gmf_mo-'+what,
    'gmf_mo_expert-'+what,
    'gmf_softmax-'+what,
    'mlp_item-'+what,
    'mlp_max-'+what,
    'mlp_indi_mean-'+what,
    'mlp_mo-'+what,
    'mlp_mo_expert-'+what,
    'mlp_softmax-'+what,
]

pd.set_option("display.precision", 4)
all_data[focus_on_data].groupby("g", as_index=False).mean().filter(regex=what+"$").T\
                .rename(columns = {0:2, 1:3, 2:4, 3:5, 4:6, 5:7, 6:8, 7:9, 8:10})\
                .style.highlight_min(color = 'lightgreen', axis = 0)


In [ ]:
print(all_data[focus_on_data].groupby("g", as_index=False).mean().filter(regex=what+"$").T.to_latex())
#T\
#                .style.highlight_min(color = 'lightgreen', axis = 0).to_latex()

In [ ]:
list(all_data.columns)

In [ ]:
# lineplot
what='MSE'
focus_on_data = [
    'g',
    #'gmf_indi_mean-'+what,
    #'gmf_mo-'+what,
    'gmf_softmax-'+what,
    'mlp_indi_mean-'+what,
    'mlp_mo-'+what,
    #'mlp_softmax-'+what,
]

processed = all_data[focus_on_data].groupby("g", as_index=False).mean()[focus_on_data].melt(['g'], value_name='error', var_name='model')
processed['model'] = processed['model'].str[:-4]

processed.loc[processed["model"] == 'gmf_softmax', "model"] = 'GMF softmax'
processed.loc[processed["model"] == 'mlp_mo', "model"] = 'MLP MultiEncod'
processed.loc[processed["model"] == 'mlp_indi_mean', "model"] = 'MLP IPA'

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,15))

# Set general font size
plt.rcParams['font.size'] = '36'
SMALL_SIZE = 32
MEDIUM_SIZE = 40
BIGGER_SIZE = 44

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE-4)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Set tick font size
#for label in (ax.get_xticklabels() + ax.get_yticklabels()):
#    label.set_fontsize(16)
    
snsfig = sns.lineplot(data=processed, x="g", y="error", hue="model")
snsfig.set(xlabel="Group Size", ylabel="Mean Squared Error")

snsfig.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
                      ncol=3, mode="expand", borderaxespad=0.)

sfig = snsfig.get_figure()
sfig.savefig(DS+'mse.png', dpi=300, transparent=True)